In [164]:
import pandas as pd
from string import ascii_uppercase as alpha
import pickle
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [ ]:
all_tables=pd.read_html('https://web.archive.org/web/20221002180521/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [ ]:
dict_tables={}
for letter,i in zip(alpha,range(11,67,7)):
    df=all_tables[i]
    df.rename(columns={df.columns[1]:'Team'},inplace=True)
    dict_tables[f"Cluster:{letter}"]=df

To csv Fixture..

In [ ]:
df=pd.concat(dict_tables)
df.to_csv('Data/Fixture_2022.csv',index=False)

Starting to filter the page

In [ ]:
years=[1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]

making a request

In [ ]:
web='https://en.wikipedia.org/wiki/2014_FIFA_World_Cup'
response=requests.get(web)

Loading text with Beatifulsoup

In [ ]:
content=response.text
soup=BeautifulSoup(content,'lxml')

Filtering rows on matches

In [ ]:
matches=soup.findAll('div',class_='footballbox')


Filtering Matches and adding to list for dataframe

In [ ]:
home=[]
score=[]
away=[]
for match in matches:
    home.append(match.find('th',class_='fhome').get_text())
    score.append(match.find('th',class_='fscore').get_text())
    away.append(match.find('th',class_='faway').get_text())
#the elements have already been added to the lists

Creating the dictionary previusly for the Dataframe

In [ ]:
dict_football={'home':home ,'score':score ,'Away':away}
df_football=pd.DataFrame(dict_football)
df_football['year']='2014'

To make the work easier, we are going to use a function, that does exactly the same as the previous thing..

In [ ]:
def get_matches(year):
    web=f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response=requests.get(web)
    content=response.text
    soup=BeautifulSoup(content,'lxml')
    matches=soup.findAll('div',class_='footballbox')
    home=[]
    score=[]
    away=[]
    for match in matches:
        home.append(match.find('th',class_='fhome').get_text())
        score.append(match.find('th',class_='fscore').get_text())
        away.append(match.find('th',class_='faway').get_text())
    #the elements have already been added to the lists
    dict_football={'home':home ,'score':score ,'Away':away}
    df_football=pd.DataFrame(dict_football)
    df_football['year']=year
    return df_football

Let's try it..

In [ ]:
#get_matches(1930)

we are going to save all the dataframes in a list

In [ ]:
fifa=[get_matches(year)for year in years]
df_fifa=pd.concat(fifa,ignore_index=True)

To csv...

In [ ]:
df_fifa.to_csv('Data/Fifa_worldcup_historical_data1.csv',index=False)

Because in certain world cups, with beautifulsoup it is not possible to obtain the data, so we will use selenium, for this especific cups

Using selenium to improve the process for collect data

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service



Importing the chromedriver

In [ ]:
service = Service('chromedriver_linux64')

Creating driver to interacts to web page

In [ ]:
driver=webdriver.Chrome(service=service)
web='https://en.wikipedia.org/wiki/1982_FIFA_World_Cup'
driver.get(web)


Getting the xpaht of the matches

In [ ]:
matches=driver.find_elements(by='xpath',value="//th[@class='fhome']/..")

Extracting the information from the score, home, away

In [ ]:
home=[]
score=[]
away=[]
for match in matches:
    home.append(match.find_element(by='xpath',value="./th[1]").text)
    score.append(match.find_element(by='xpath',value="./th[2]").text)
    away.append(match.find_element(by='xpath',value="./th[3]").text)

Creating a Dataframe..

In [ ]:
dict_football2={'home':home,'score':score,'away':away}
df_history_worldcup=pd.DataFrame(dict_football2)
df_history_worldcup.to_csv('World_cup_1982.csv',index=False)


Now we are going to apply all that in a function that goes through all the world cups and prints it in a csv....

In [ ]:
def get_all_dataworldcup(year):
    service = Service('chromedriver_linux64')
    driver=webdriver.Chrome(service=service)
    web=f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    driver.get(web)
    matches=driver.find_elements(by='xpath',value="//th[@class='fhome']/..")
    home=[]
    score=[]
    away=[]
    for match in matches:
        home.append(match.find_element(by='xpath',value="./th[1]").text)
        score.append(match.find_element(by='xpath',value="./th[2]").text)
        away.append(match.find_element(by='xpath',value="./th[3]").text)
    dict_football2={'home':home,'score':score,'away':away}
    df_history_worldcup=pd.DataFrame(dict_football2)
    df_history_worldcup['year']=year
    time.sleep(2)
    return df_history_worldcup
#this element is out of a function because
fifa2=[get_all_dataworldcup(year) for year in years]
df_fifa2=pd.concat(fifa2,ignore_index=True)
df_fifa2.to_csv('Data/Fifa_worldcup_historical_data1.csv',index=False)
driver.quit()

Now Clean data with Pandas!

In [165]:
history=pd.read_csv('Data/Fifa_worldcup_missing_data.csv',)
fixture=pd.read_csv('Data/Fixture_2022.csv')


Missing data..

In [166]:
fixture['home']=fixture['home'].apply(lambda x: x.strip())
fixture['away']=fixture['away'].str.strip()

Drop Null..

In [167]:
history[history['home'].isnull()]
history.drop_duplicates(inplace=True)
history.sort_values('year',inplace=True)

As I am going to predict based on the goals, then the matches that were assigned arbitrarily I have to eliminate them for example the match between Sweden and Austria..

In [168]:
index_element_todrop=history[history['home'].str.contains('Sweden') & history['away'].str.contains('Austria')].index
history.drop(index=index_element_todrop,inplace=True,)
history

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
863,Brazil,2–0,Costa Rica,2018
864,Serbia,1–2,Switzerland,2018
865,Serbia,0–2,Brazil,2018
867,Germany,0–1,Mexico,2018


There are many matches that have text data in the score, let's clean that up...

In [169]:
# history[history['score'].str.contains('(a.e.t.)')]
history['score']=history['score'].str.replace('[^\d–\d$]]','',regex=True)
history

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
863,Brazil,2–0,Costa Rica,2018
864,Serbia,1–2,Switzerland,2018
865,Serbia,0–2,Brazil,2018
867,Germany,0–1,Mexico,2018


Now clean the dataframe history..

In [170]:
history['home']=history['home'].str.strip()
history['away']=history['away'].str.strip()

Now we are going to split the score column in 2, to train the model.

In [175]:
history[['HomeGoals','AwayGoals']]=history['score'].str.split('–',expand=True)
history.drop('score',axis=1)

,home,score,away,year,HomeGoals,AwayGoals
0,France,4–1,Mexico,1930,4,1
17,Uruguay,4–2,Argentina,1930,4,2
16,Uruguay,6–1,Yugoslavia,1930,6,1
15,Argentina,6–1,United States,1930,6,1
14,Paraguay,1–0,Belgium,1930,1,0
...,...,...,...,...,...,...
863,Brazil,2–0,Costa Rica,2018,2,0
864,Serbia,1–2,Switzerland,2018,1,2
865,Serbia,0–2,Brazil,2018,0,2
867,Germany,0–1,Mexico,2018,0,1
